In [1]:
import numpy as np
import pandas as pd
import pulp as lp
import pickle as pkl
import time
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt
import ALMPlanner as ALM
import ALMChart as ALMc

In [2]:
######
EX = 1
######


if EX == 1:
    problem = ALM.ALMplanner(start = "Jan 2021", end = "Jan 2041", user_risk_profile = 0)
    # set planned liabilities
    problem.liabilities.insert("car", "Jan 2026", 30000, 30000*0.65)
    problem.liabilities.insert("university", "Jan 2029", 50000, 50000*0.95)
    problem.liabilities.insert("hawaii", "Jan 2037",30000, 30000*0.85)
    # set planned assets 
    problem.assets.insert("ass_0","Jan 2021",30000)
    recurrent_dates = ["Jan 2022", "Jan 2023", "Jan 2024", "Jan 2025", "Jan 2026", "Jan 2027"]
    for i in np.arange(len(recurrent_dates)):
        problem.assets.insert("ass_" + str(i+1),recurrent_dates[i],10000)
elif EX == 2:
    problem = ALM.ALMplanner(start = "Jan 2021", end = "Jan 2061")
    # set planned liabilities
    recurrent_assets = pd.date_range(start = "Jan 2021", end="Jan 2040", freq = pd.offsets.YearBegin(1))
    for i in np.arange(len(recurrent_assets)):
        problem.assets.insert("ass_" + str(i+1),recurrent_assets[i],1000)
    recurrent_liabilities = pd.date_range(start = "Jan 2041", end="Jan 2060", freq = pd.offsets.YearBegin(1))
    for i in np.arange(len(recurrent_liabilities)):
        problem.liabilities.insert("liab_" + str(i+1),recurrent_liabilities[i],1500,1100)

In [3]:
ALMc.display(problem,bar_width = 6)

In [4]:
problem.check_feasibility()
problem.get_feasibility()

check feasibility ended in 10.29 s with Optimal solution
Unfeasible problem: suggested +2.53%  on assets


In [5]:
ALMc.display(problem,bar_width = 6)

In [6]:
GB_model = ALM.ALMGoalBased(problem)
GB_model.solve(problem)

BaH_model = ALM.ALMBuyAndHold(problem)
BaH_model.solve(problem)

GoalBased model generated in 9.08 s
Solve ended in 1.55 s with Optimal solution
BuyAndHold model generated in 8.98 s
Solve ended in 0.63 s with Optimal solution


# Display solution

In [17]:
sol = GB_model.solution
sol_bah = BaH_model.solution


In [7]:
P = problem.P
A = problem.assets.set
L = problem.liabilities.set
N = problem.N
T = problem.T
Scenario = problem.Scenario

In [34]:
def AssetSplitDetailsChart(planner, solution, groupby): # Da sistemare
    P = planner.P
    A = planner.assets.set
    L = planner.liabilities.set
    N = planner.N
    T = planner.T
    Assets_split = pd.DataFrame(index = np.arange(len(P)*len(A)*(len(L)+1)), columns = ["Asset", "Goal", "ETF", "Value"])
    iter = -1
    for a in A:
        for p in P:
            iter = iter+1
            Assets_split["Asset"][iter] = a
            Assets_split["Goal"][iter] = "extra_wealth"
            Assets_split["ETF"][iter] = p
            Assets_split["Value"][iter] = solution.asset_to_exwealth[a][p]
            for l in L:
                iter = iter+1
                Assets_split["Asset"][iter] = a
                Assets_split["Goal"][iter] = l
                Assets_split["ETF"][iter] = p
                Assets_split["Value"][iter] = solution.asset_to_goal[a][l][p]
    print(Assets_split)
    AssetGroupedBy = Assets_split[["Asset", groupby, "Value"]].groupby(by=[groupby], axis = 1).sum()
    print(AssetGroupedBy)
    #AssetPerLiab_melt = AssetPerLiab.melt(id_vars=['index'], var_name='Liab', value_name='split')
    #print(AssetPerLiab_melt.head())
    #plt = px.bar(AssetPerLiab_melt, x="index", y = "split", color = "Liab")
    #plt = ALMc.standardized_chart(plt)
    
AssetSplitDetailsChart(problem, sol, "ETF")

     Asset          Goal       ETF      Value
0    ass_0  extra_wealth      Cash        0.0
1    ass_0           car      Cash  288.59007
2    ass_0    university      Cash  14428.228
3    ass_0        hawaii      Cash        0.0
4    ass_0  extra_wealth  GVT_EU13        0.0
..     ...           ...       ...        ...
163  ass_6        hawaii     EQ_EU        0.0
164  ass_6  extra_wealth     EQ_US        0.0
165  ass_6           car     EQ_US        0.0
166  ass_6    university     EQ_US        0.0
167  ass_6        hawaii     EQ_US        0.0

[168 rows x 4 columns]
Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99

In [21]:
def AssetSplitDetailsChart(planner, solution, groupby):
    P = planner.P
    A = planner.assets.set
    L = planner.liabilities.set
    N = planner.N
    T = planner.T
    Assets_l = {}
    Assets_end = {}
    for a in A:
        Assets_l[a] = np.zeros(shape = (len(L), len(P)))
        Assets_end[a] = np.zeros(shape = (len(P)))
        for p in np.arange(len(P)):
            Assets_end[a][p] = solution.asset_to_exwealth[a][P[p]]
            for l in np.arange(len(L)):
                Assets_l[a][l,p] = solution.asset_to_goal[a][L[l]][P[p]]

    #AssetPerETF
    AssetPerETF = {}

    for a in A:
        Asset_ap = np.sum(Assets_l[a], axis = 0) + Assets_end[a] 
        AssetPerETF[a] = {P[p]:Asset_ap[p] for p in np.arange(len(P))}

    AssetPerETF = pd.DataFrame(AssetPerETF).transpose().reset_index()
    AssetPerETF_melt = AssetPerETF.melt(id_vars=['index'], var_name='P', value_name='split')
    plt = px.bar(AssetPerETF_melt, x="index", y = "split", color = "P" )
    plt = ALMc.standardized_chart(plt)
    print(AssetPerETF_melt.head())
    #AssetPerETF.plot.bar(stacked = True, figsize = (15,10))
    plt.show()
    AssetPerLiab = {}
    for a in A:
        Asset_al = list(np.sum(Assets_l[a], axis = 1))
        Asset_al.append(np.sum(Assets_end[a]))
        L_end = list(L)
        L_end.append("end")
        AssetPerLiab[a] = {L_end[l]:Asset_al[l] for l in np.arange(len(L_end))}

    AssetPerLiab = pd.DataFrame(AssetPerLiab).transpose().reset_index()
    AssetPerLiab_melt = AssetPerLiab.melt(id_vars=['index'], var_name='Liab', value_name='split')
    print(AssetPerLiab_melt.head())
    plt = px.bar(AssetPerLiab_melt, x="index", y = "split", color = "Liab")
    plt = ALMc.standardized_chart(plt)
    plt.show()

AssetSplitDetailsChart(problem, sol, 0)

   index     P        split
0  ass_0  Cash  14716.81807
1  ass_1  Cash   1567.35611
2  ass_2  Cash   8135.28220
3  ass_3  Cash   9240.39860
4  ass_4  Cash  10243.99900


   index Liab         split
0  ass_0  car    651.817491
1  ass_1  car   1461.138420
2  ass_2  car     24.914615
3  ass_3  car   7206.619563
4  ass_4  car  10253.205163


In [11]:

plt.show()

index         ass_0ass_1ass_2ass_3ass_4ass_5ass_6
car                                  19597.695252
university                           47979.226206
hawaii                                24701.92926
end                                           0.0
dtype: object


In [18]:
n_scen = None
perc = False
#print([solution[l][n_scen].varValue for l in Lt])
plt = ALMc.AssetAllocationChart(problem,sol,n_scen=n_scen, perc = False)
plt.show()
plt = ALMc.AssetAllocationChart(problem,sol_bah,n_scen=n_scen, perc = False)
plt.show()

In [ ]:
Q_ln = {}
for l in L:
    Q_ln[l] = np.zeros(shape = (len(N)))
    for n in N:
        Q_ln[l][n]=sol.goal_distr[l][N[n]]

Q_ln["ex_wealth"] = np.zeros(shape = (len(N)))
for n in N:
    Q_ln["ex_wealth"][n]=sol.final_exwealth[N[n]]

df_Q_ln = pd.DataFrame(Q_ln)

In [ ]:
conf_tg = pd.DataFrame(problem.liabilities.value_tg, columns = problem.liabilities.set, index = N)
conf_lb = pd.DataFrame(problem.liabilities.value_lb, columns = problem.liabilities.set, index = N)
sub_goal_prob = np.logical_and(df_Q_ln[problem.liabilities.set] < conf_tg, df_Q_ln[problem.liabilities.set] >= conf_lb).mean()
fail_goal_prob = (df_Q_ln[problem.liabilities.set] < conf_lb).mean()
print(sub_goal_prob, fail_goal_prob)